In [47]:
from os import environ

from sqlalchemy import Connection, create_engine, sql
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv

load_dotenv()

True

In [48]:
def get_database_connection() -> Connection:
    """
    Establishes a database connection to the database specified.
    """

    try:
        engine = create_engine(
            f"mssql+pymssql://{environ['DB_USER']}:{environ['DB_PASSWORD']}@{environ['DB_HOST']}/?charset=utf8")

        return engine.connect()

    except ConnectionError as error:
        print(error)


conn = get_database_connection()

In [49]:
def load_all_plant_data(conn: Connection) -> DataFrame:

    conn.execute(sql.text("USE plants;"))

    query = sql.text(
        """SELECT plant_condition.plant_condition_id,
        plant_condition.at, plant_condition.soil_moisture,
        plant_condition.temperature, plant_condition.last_watered,
        plant.plant_id, plant.plant_name,
        plant.scientific_name, botanist.botanist_id, botanist.first_name,
        botanist.surname, botanist.email, botanist.phone_number,
        origin.origin_id, origin.latitude, origin.longitude, origin.region
        FROM s_epsilon.plant_condition
        JOIN s_epsilon.plant ON s_epsilon.plant.plant_id = s_epsilon.plant_condition.plant_id
        JOIN s_epsilon.botanist ON s_epsilon.botanist.botanist_id = s_epsilon.plant.botanist_id
        JOIN s_epsilon.origin ON s_epsilon.origin.origin_id = s_epsilon.plant.origin_id;""")

    conn.execute(sql.text("COMMIT;"))
    res = conn.execute(query).fetchall()
    df = pd.DataFrame(res)

    return df

df = load_all_plant_data(conn)

df.head(45)

,plant_condition_id,at,soil_moisture,temperature,last_watered,plant_id,plant_name,scientific_name,botanist_id,first_name,surname,email,phone_number,origin_id,latitude,longitude,region
0,2,2023-12-19 11:00:45,25.222584,12.013141,2023-12-18 13:54:32,1,Venus flytrap,N/A,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,2,33.95015,-118.03917,South Whittier
1,3,2023-12-19 11:00:46,31.762604,9.081111,2023-12-18 14:10:54,2,Corpse flower,N/A,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,3,7.65649,4.92235,Efon-Alaaye
2,4,2023-12-19 11:00:48,30.228126,9.959602,2023-12-18 14:50:16,3,Rafflesia arnoldii,N/A,3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,1,-19.32556,-41.25528,Resplendor
3,5,2023-12-19 11:00:49,22.201815,11.305301,2023-12-18 13:16:25,4,Black bat flower,N/A,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,4,13.70167,-89.10944,Ilopango
4,6,2023-12-19 11:00:50,31.625155,11.154798,2023-12-18 14:56:18,5,Pitcher plant,Sarracenia catesbaei,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,5,22.88783,84.13864,Jashpurnagar
5,7,2023-12-19 11:00:51,28.529411,10.943492,2023-12-18 13:35:35,6,Wollemi pine,Wollemia nobilis,3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,6,43.86682,-79.26630,Markham
6,8,2023-12-19 11:00:52,22.593150,11.482004,2023-12-18 13:23:01,8,Bird of paradise,Heliconia schiedeana 'Fire and Ice',3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,7,5.27247,-3.59625,Bonoua
7,9,2023-12-19 11:00:52,23.861438,13.930351,2023-12-18 13:04:57,10,Dragon tree,N/A,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,9,43.50891,16.43915,Split
8,10,2023-12-19 11:00:53,27.893563,9.432708,2023-12-18 13:37:24,11,Asclepias Curassavica,Asclepias curassavica,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,10,20.88953,-156.47432,Kanului
9,11,2023-12-19 11:00:55,22.962426,12.752440,2023-12-18 13:13:51,12,Brugmansia X Candida,N/A,3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,11,32.50070,-94.74049,Longview


## Average soil moisture

In [50]:
df["soil_moisture"] = df["soil_moisture"].astype(float)

average_soil_moisture = df["soil_moisture"].mean()

average_soil_moisture

27.602242228585837